In [2]:
pip install numpy tensorflow opencv-python Pillow pickle5


     ---------------------------------------- 0.0/132.1 kB ? eta -:--:--
     ----- ------------------------------- 20.5/132.1 kB 330.3 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/132.1 kB 544.7 kB/s eta 0:00:01
     -------------------------------------- 132.1/132.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pickle5
Failed to build pickle5
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [17 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-312
  creating build\lib.win-amd64-cpython-312\pickle5
  copying pickle5\pickle.py -> build\lib.win-amd64-cpython-312\pickle5
  copying pickle5\pickletools.py -> build\lib.win-amd64-cpython-312\pickle5
  copying pickle5\__init__.py -> build\lib.win-amd64-cpython-312\pickle5
  creating build\lib.win-amd64-cpython-312\pickle5\test
  copying pickle5\test\pickletester.py -> build\lib.win-amd64-cpython-312\pickle5\test
  copying pickle5\test\test_pickle.py -> build\lib.win-amd64-cpython-312\pickle5\test
  copying pickle5\test\test_picklebuffer.py -> build\lib.win-amd64-cpython-312\pickle5\test
  copying pickle5\test\__init__.py -> build\lib.win-amd64-cpython-312\pickle5\test
  running build_ext
  building 'pickle5._pickle' extension
  error: Mi

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pickle


In [13]:
# Define the paths for your dataset
train_directory = r'C:\Users\kshit\Downloads\archive\Dataset\Train'
validation_directory = r'C:\Users\kshit\Downloads\archive\Dataset\Validation'
test_directory = r'C:\Users\kshit\Downloads\archive\Dataset\Test'

# ImageDataGenerator for loading images from the train, validation, and test directories
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow images in batches from the directory for training
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'  # Assuming binary classification: 'real' and 'fake'
)

# Flow images in batches from the directory for validation
validation_generator = validation_datagen.flow_from_directory(
    validation_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Flow images in batches from the directory for testing
test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Define the model (already done earlier in the code)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


In [14]:
# Early stopping and model checkpoint to save the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping, checkpoint]
)

# Evaluate the model on the test dataset
evaluation = model.evaluate(test_generator)
print(f"Test Accuracy: {evaluation[1]*100:.2f}%")

Epoch 1/10


C:\Users\kshit\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 716ms/step - accuracy: 0.8111 - loss: 0.3987
Epoch 1: val_accuracy improved from -inf to 0.83512, saving model to best_model.keras
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 4055s 925ms/step - accuracy: 0.8111 - loss: 0.3987 - val_accuracy: 0.8351 - val_loss: 0.3669
Epoch 2/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 38us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


C:\Users\kshit\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
C:\Users\kshit\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
C:\Users\kshit\anaconda3\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 525ms/step - accuracy: 0.8559 - loss: 0.3232
Epoch 3: val_accuracy improved from 0.83512 to 0.84260, saving model to best_model.keras
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 2936s 671ms/step - accuracy: 0.8559 - loss: 0.3232 - val_accuracy: 0.8426 - val_loss: 0.3556
Epoch 4/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 19us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 529ms/step - accuracy: 0.8665 - loss: 0.3017
Epoch 5: val_accuracy did not improve from 0.84260
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 2958s 676ms/step - accuracy: 0.8665 - loss: 0.3017 - val_accuracy: 0.8424 - val_loss: 0.3510
Epoch 6/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 24us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 0s 567ms/step - accuracy: 0.8848 - loss: 0.2685
Epoch 7: val_accuracy improved from 0.84260 to 0.85160, saving model to best_model.keras
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 3146s 719ms/step - accuracy: 0.8848 - l

In [16]:

# Save the final model weights
model.save('final_deepfake_model.h5')



In [31]:
# Evaluate the model on the test set
test_generator = validation_datagen.flow_from_directory(
    test_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")



Found 10905 images belonging to 2 classes.
341/341 ━━━━━━━━━━━━━━━━━━━━ 174s 511ms/step - accuracy: 0.7440 - loss: 0.7213
Test Accuracy: 74.29%


In [35]:
# Saving the model architecture and weights using Pickle
with open('deepfake_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Saving the model weights separately using the correct .weights.h5 extension
model.save_weights('deepfake_model_weights.weights.h5')  # Changed to .weights.h5

# Loading the model and weights in the future
# with open('deepfake_model.pkl', 'rb') as file:
#     loaded_model = pickle.load(file)
# loaded_model.load_weights('deepfake_model_weights.weights.h5')  # Load the .weights.h5 weights